<a href="https://colab.research.google.com/github/tho071206-ther/TriTueNhanTao/blob/main/alphabeta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bai thuc hanh tuan 3

In [2]:
import math
import sys

# giới hạn đệ quy
sys.setrecursionlimit(2000)

NGUOI_CHOI_MAX = 'X' # Người chơi MAX (AI) - Luôn tối đa hóa điểm
DOI_THU_MIN = 'O'    # Người chơi MIN (Đối thủ) - Luôn tối thiểu hóa điểm
DIEM_THANG = 1000000 # Điểm rất cao cho trạng thái thắng (utility)


def get_board_size():
    """Nhập kích thước N của bàn cờ N*N (N >= 3)."""
    while True:
        try:
            n = int(input("NHẬP: Kích thước bàn cờ N*N (ví dụ 3, 5, 10): "))
            if n >= 3:
                return n
            else:
                print("LỖI! Vui lòng nhập N >= 3!")
        except ValueError:
            print("LỖI! Vui lòng nhập một số nguyên!")

def check_win(ban_co, nguoi_choi):
    """
    Kiểm tra xem 'nguoi_choi' có đạt N quân liên tiếp trên bàn cờ N*N hay không.
    """
    N = len(ban_co)

    # 1. Kiểm tra Ngang (Hàng)
    for r in range(N):
        if all(ban_co[r][c] == nguoi_choi for c in range(N)):
            return True

    # 2. Kiểm tra Dọc (Cột)
    for c in range(N):
        if all(ban_co[r][c] == nguoi_choi for r in range(N)):
            return True

    # 3. Kiểm tra Chéo Chính (\)
    if all(ban_co[i][i] == nguoi_choi for i in range(N)):
        return True

    # 4. Kiểm tra Chéo Phụ (/)
    if all(ban_co[i][N - 1 - i] == nguoi_choi for i in range(N)):
        return True

    return False

def is_terminal(ban_co):
    """Kiểm tra xem bàn cờ đã kết thúc (thắng, thua, hòa) hay vẫn tiếp tục."""
    if check_win(ban_co, NGUOI_CHOI_MAX) or check_win(ban_co, DOI_THU_MIN):
        return True # Trạng thái Thắng/Thua

    # Kiểm tra Hòa (hết ô trống)
    for hang in ban_co:
        if ' ' in hang:
            return False
    return True # Bàn cờ đầy, trạng thái Hòa

def evaluate(ban_co):
    """Hàm đánh giá lợi ích (Utility) khi đạt đến trạng thái kết thúc."""
    if check_win(ban_co, NGUOI_CHOI_MAX):
        return DIEM_THANG     # AI thắng
    if check_win(ban_co, DOI_THU_MIN):
        return -DIEM_THANG   # Đối thủ thắng
    return 0                 # Hòa

def get_available_moves(ban_co):
    """Tìm tất cả các nước đi hợp lệ (ô trống)."""
    moves = []
    N = len(ban_co)
    for hang in range(N):
        for cot in range(N):
            if ban_co[hang][cot] == ' ':
                moves.append((hang, cot))
    return moves


def minimax_alpha_beta(ban_co, do_sau, alpha, beta, la_luot_toi_da_hoa):
    """
    Hàm Minimax đệ quy với Cắt tỉa Alpha-Beta.

    - alpha: Giá trị tốt nhất MAX có thể đảm bảo (Lower bound).
    - beta: Giá trị tốt nhất MIN có thể đảm bảo (Upper bound).
    """

    # ĐIỀU KIỆN DỪNG: Đạt độ sâu 0 HOẶC Trạng thái kết thúc
    if do_sau == 0 or is_terminal(ban_co):
        return evaluate(ban_co)

    if la_luot_toi_da_hoa:
        # LƯỢT CỦA MAX (AI) - Tối đa hóa
        gia_tri_toi_da = -math.inf

        for hang, cot in get_available_moves(ban_co):
            # Thử nước đi ảo
            ban_co[hang][cot] = NGUOI_CHOI_MAX

            # Gọi đệ quy: chuyển sang lượt MIN (tối thiểu hóa)
            gia_tri = minimax_alpha_beta(ban_co, do_sau - 1, alpha, beta, False)

            # Hoàn trả trạng thái (Backtrack)
            ban_co[hang][cot] = ' '

            # Cập nhật giá trị và Alpha
            gia_tri_toi_da = max(gia_tri_toi_da, gia_tri)
            alpha = max(alpha, gia_tri_toi_da)

            # CẮT TỈA BETA (Nếu max_eval >= beta)
            # MAX tìm thấy giá trị tốt hơn giá trị đảm bảo của MIN
            # Cắt bỏ MIN
            if gia_tri_toi_da >= beta:
                break

        return gia_tri_toi_da

    else:
        # LƯỢT CỦA MIN (Đối thủ) - Tối thiểu hóa
        gia_tri_toi_thieu = math.inf

        for hang, cot in get_available_moves(ban_co):
            # Thử nước đi ảo
            ban_co[hang][cot] = DOI_THU_MIN

            # Gọi đệ quy: chuyển sang lượt MAX (tối đa hóa)
            gia_tri = minimax_alpha_beta(ban_co, do_sau - 1, alpha, beta, True)

            # Hoàn trả trạng thái (Backtrack)
            ban_co[hang][cot] = ' '

            # Cập nhật giá trị và Beta
            gia_tri_toi_thieu = min(gia_tri_toi_thieu, gia_tri)
            beta = min(beta, gia_tri_toi_thieu)

            # CẮT TỈA ALPHA (Nếu min_eval <= alpha)
            # MIN tìm thấy giá trị tồi tệ hơn giá trị đảm bảo của MAX
            # Cắt bỏ Max
            if gia_tri_toi_thieu <= alpha:
                break

        return gia_tri_toi_thieu


def find_best_move_alpha_beta(ban_co, do_sau):
    """Hàm tìm nước đi tối ưu nhất cho AI (MAX) bằng Alpha-Beta."""
    best_eval = -math.inf
    best_move = None

    # Khởi tạo giá trị alpha và beta cho node gốc
    alpha = -math.inf
    beta = math.inf

    for move in get_available_moves(ban_co):
        hang, cot = move

        ban_co[hang][cot] = NGUOI_CHOI_MAX # Thử nước đi


        # Node con của gốc là node MIN nhận alpha và beta của node gốc.
        eval = minimax_alpha_beta(ban_co, do_sau - 1, alpha, beta, False)

        ban_co[hang][cot] = ' ' # Hoàn trả trạng thái

        # Cập nhật nước đi tốt nhất
        if eval > best_eval:
            best_eval = eval
            best_move = move

        # Cập nhật alpha của node gốc (node MAX)
        alpha = max(alpha, best_eval)

    return best_move, best_eval



if __name__ == '__main__':
    N = get_board_size()
    ban_co = [[' ' for _ in range(N)] for _ in range(N)]

    # Độ sâu tìm kiếm

    search_depth = min(4, N*N)

    print("\n--- THUẬT TOÁN MINIMAX CÓ CẮT TỈA ALPHA-BETA ---")
    print(f"Kích thước: {N}x{N}, Độ sâu tìm kiếm: {search_depth}")

    print(f"\n[AI đang suy nghĩ với Alpha-Beta (Độ sâu = {search_depth})]...")

    # Bắt đầu tìm kiếm nước đi tối ưu
    best_move, score = find_best_move_alpha_beta(ban_co, search_depth)

    if best_move:
        hang, cot = best_move
        # Thực hiện nước đi lên bàn cờ thật
        ban_co[hang][cot] = NGUOI_CHOI_MAX

        print(f"-> Nước đi tối ưu (Alpha-Beta): ({hang}, {cot})")
        print(f"-> Giá trị thế cờ (Score): {score}")

        # In bàn cờ
        print("\n--- TRẠNG THÁI BÀN CỜ SAU NƯỚC ĐI CỦA AI ---")
        N = len(ban_co)
        print("   " + " ".join(str(i) for i in range(N)))
        print("  " + "---" * N)
        for i in range(N):
            print(f"{i} | " + " ".join(ban_co[i]))
        print("  " + "---" * N)

    else:
        print("Không còn nước đi nào.")

NHẬP: Kích thước bàn cờ N*N (ví dụ 3, 5, 10): 3

--- THUẬT TOÁN MINIMAX CÓ CẮT TỈA ALPHA-BETA ---
Kích thước: 3x3, Độ sâu tìm kiếm: 4

[AI đang suy nghĩ với Alpha-Beta (Độ sâu = 4)]...
-> Nước đi tối ưu (Alpha-Beta): (0, 0)
-> Giá trị thế cờ (Score): 0

--- TRẠNG THÁI BÀN CỜ SAU NƯỚC ĐI CỦA AI ---
   0 1 2
  ---------
0 | X    
1 |      
2 |      
  ---------
